In [1]:
import sys
sys.path.append("..")
from models.CTTS import CTTS

import torch 
import pandas as pd
import torch.optim as optim

In [2]:
# load train and test data here

In [3]:
if torch.cuda.is_available():
    device = torch.device('cuda')          # Use the first available GPU
    # device = torch.device('cuda:0')      # To specify a particular GPU
else:
    device = torch.device('cpu')

print(f"Using Device: {device}")

Using Device: cpu


In [4]:
model = CTTS(device=device)

/home/holden/.conda/envs/pytorch_env/lib/python3.12/site-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [ ]:
learning_rate = 1e-3
optimizer = optim.AdamW(params=model.parameters() ,lr=learning_rate)

batch_size = 64
max_epochs = 100